In [25]:

%matplotlib inline
from ipywidgets import interact
import pandas as pd
import logging
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('dark_background')
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
import category_encoders as ce

In [22]:
df = pd.read_csv('train_features.csv')
print(df.shape)
df.describe()

(37745, 103)


,id,member_id,loan_amnt,funded_amnt,installment,annual_inc,url,desc,dti,delinq_2yrs,...,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
count,37745.000000,0.0,37745.000000,37745.000000,37745.000000,3.774500e+04,0.0,0.0,37653.000000,37745.000000,...,4738.000000,4738.000000,4738.000000,4738.000000,4639.000000,4738.000000,4738.000000,4738.000000,4738.000000,1668.000000
mean,33579.507193,NaN,14913.304411,14913.304411,444.702933,7.962921e+04,NaN,NaN,18.508652,0.236111,...,33654.305192,0.724567,1.736809,11.296117,55.261199,2.855424,12.897425,0.048333,0.073660,36.332134
std,13609.172414,NaN,10153.364841,10153.364841,293.571158,8.052291e+04,NaN,NaN,19.670753,0.749590,...,29585.734663,1.068437,1.867062,6.557512,26.967270,3.054911,8.512730,0.455754,0.330458,24.058143
min,10000.000000,NaN,1000.000000,1000.000000,30.120000,0.000000e+00,NaN,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,21824.000000,NaN,7000.000000,7000.000000,222.990000,4.700000e+04,NaN,NaN,10.320000,0.000000,...,13968.500000,0.000000,0.000000,7.000000,35.350000,1.000000,7.000000,0.000000,0.000000,15.000000
50%,33603.000000,NaN,12000.000000,12000.000000,361.500000,6.700000e+04,NaN,NaN,16.680000,0.000000,...,25657.000000,0.000000,1.000000,10.000000,57.300000,2.000000,11.000000,0.000000,0.000000,34.000000
75%,45357.000000,NaN,20000.000000,20000.000000,610.170000,9.500000e+04,NaN,NaN,23.930000,0.000000,...,43273.500000,1.000000,3.000000,15.000000,76.900000,4.000000,17.000000,0.000000,0.000000,57.000000
max,57181.000000,NaN,40000.000000,40000.000000,1587.230000,9.300000e+06,NaN,NaN,999.000000,19.000000,...,290237.000000,6.000000,15.000000,58.000000,163.900000,35.000000,75.000000,20.000000,7.000000,125.000000


In [23]:

def clean(dat: pd.DataFrame) -> pd.DataFrame: 
    ''' refactored Ryan H's wrangle function from lecture today into method chaining'''
    todrop = ['id', # id is random
              'member_id', # all null
              'url', # all null
              'desc', # all null
              'title', # duplicate of purpose
              'grade', # duplicate of sub_grade
              'emp_title', # getting re-engineered, cardinality too high
              'zip_code' # cardinality too high
             ]

    def wrangle_sub_grade(x):
        '''Transform sub_grade from "A1" - "G5" to 1.1 - 7.5'''
        first_digit = ord(x[0]) - 64
        second_digit = int(x[1])
        return first_digit + second_digit/10
    
    assigns = {# sub_grade to ordinal
        **{'sub_grade': dat.sub_grade.apply(wrangle_sub_grade)}, # sub_grade to ordinal
        # Convert percentages from strings to floats
        **{name: dat[name].str.strip('%').astype(float) 
                 for name in ['int_rate', 'revol_util']}, # Convert percentages from strings to floats
        # Transform earliest_cr_line to an integer: how many days it's been open
        **{'earliest_cr_line': (pd.Timestamp.today() - \
                                  pd.to_datetime(dat.earliest_cr_line, infer_datetime_format=True)
                               ).dt.days},  
        # Create features for three employee titles: teacher, manager, owner
        **{'emp_title_'+name: dat.emp_title.str.contains(name, na=False) 
                              for name in ['teacher', 'manager', 'owner']} 
              }

    return dat.assign(emp_title = dat.emp_title.str.lower()).assign(**assigns).drop(todrop, axis=1)
    
df = clean(pd.read_csv('train_features.csv').sample(2000))

df.head()

,loan_amnt,funded_amnt,term,int_rate,installment,sub_grade,emp_length,home_ownership,annual_inc,purpose,...,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method,emp_title_teacher,emp_title_manager,emp_title_owner
30040,12000,12000,60 months,11.99,266.88,2.5,10+ years,MORTGAGE,47700.0,home_improvement,...,NaN,NaN,NaN,NaN,NaN,NaN,Cash,False,False,False
9613,5000,5000,36 months,10.41,162.31,2.3,1 year,MORTGAGE,60000.0,debt_consolidation,...,NaN,NaN,NaN,NaN,NaN,NaN,Cash,False,False,False
27913,9500,9500,36 months,10.91,310.62,2.4,10+ years,MORTGAGE,120000.0,debt_consolidation,...,NaN,NaN,NaN,NaN,NaN,NaN,Cash,False,False,False
9959,15000,15000,36 months,15.04,520.28,3.4,10+ years,MORTGAGE,19000.0,debt_consolidation,...,64.9,2.0,9.0,0.0,0.0,NaN,Cash,False,False,False
10598,40000,40000,60 months,11.55,880.71,2.4,10+ years,MORTGAGE,120000.0,small_business,...,NaN,NaN,NaN,NaN,NaN,NaN,Cash,False,False,False
